<h2>Before you start</h2>
If this is the first time the pipeline is running on this machine, just run the cell below. It will copy startup.py from the BEARMIND folder into your local startup folder. This allows the code in startup.py to be executed automatically after each kernel restart (and removes the need to monotonously click through all setup cells after each reloading).

In [ ]:
import shutil
import os

local_startup_dir = get_ipython().profile_dir.startup_dir
filedir = os.getcwd()
shutil.copy(os.path.join(filedir, 'startup.py'), os.path.join(local_startup_dir, 'startup.py'))
%run startup.py

<h2>Module 0</h2>
You need to specify the root folder and pathway pattern. Note that * is a wildcard for any symbol combination except slashes (i.e., for any folder name), so it is strongly recommended to use it here.<br/><br/>
NB!! Just in case, use double backslashes for folder separation, otherwise some symbols may be interpreted as escape sequences. 

In [ ]:
root = '/export/home1/Test_min_data/'
data_path_pattern = '*/*/Miniscope/'

<h2>Module 1</h2>
Manual video inspection. <br/>Open folder with miniscopic videos in a pop-up window, wait for loading and specify margins to be cropped by sliders or by keyboard, then save them by running the next cell. At the time, cropping .pickle files are to be created in these folders. Repeat for all folders with miniscopic videos you would like to analyze.   

In [ ]:
#Manual file selection:
fnames = list(askopenfilenames(title = 'Select files for inspection', initialdir = root, filetypes = [('AVI files', '.avi')]))
#OR, alternatively, you can use Automatic file selection
#fnames = glob(path + data_path_pattern + '*.avi')

data = LoadSelectedVideos(fnames)
w = DrawCropper(data, fname=fnames[0])

Batch cropping and timestamp extraction.<br/>Miniscopic videos from folders with .pickle files are to be cropped and saved as _CR.tif in the root folder. There is no need for renaming of sigle-digit .avi files (like 0-9.avi to 00-09.avi)!<br/>
Also, along with video data, timestamps are to be copied from minicopic folders to the root folder. Do not delete them, they are nessesary for the further steps!

In [ ]:
#Batch crop
pick_names = glob(root + data_path_pattern + '*cropping.pickle')

for name in pick_names:
    out_name = name[len(root): len(os.path.dirname(name))].replace(os.path.sep, '_') 
    DoCropAndRewrite(name, root + out_name + '.tif')
    #copy timestamps from Miniscope folder to the root folder
    tst_name = os.path.join(os.path.dirname(name), 'timeStamps.csv')
    shutil.copy(tst_name, root + out_name + '_timestamp.csv')


<h2>Module 2</h2>
Batch motion correction.<br/>All _CR.tif files in the root folder are to be automatically motion corrected with NoRMCorre routine [Pnevmatikakis, Giovanucci, 2017] with the parameters below and saved as _MC.tif files.

In [ ]:
#Automatic file selection
#fnames = glob(root + data_path_pattern + '*_CR.tif')
#OR, alternatively, you can use manual file selection:
fnames = askopenfilenames(title = 'Select files for motion correction', initialdir = root, filetypes = [('TIFF files', '.tif')])

mc_dict = {
    'pw_rigid': False,         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
    'max_shifts': (35, 35),    # maximum allowed rigid shift
    'gSig_filt': (8, 8),       # size of high pass spatial filtering, used in 1p data
    'strides': (48, 48),       # start a new patch for pw-rigid motion correction every x pixels
    'overlaps': (24, 24),      # overlap between pathes (size of patch strides+overlaps)
    'max_deviation_rigid': 15,  # maximum deviation allowed for patch with respect to rigid shifts
    'border_nan': 'copy',      # replicate values along the boundaries
    'use_cuda': True,          # Set to True in order to use GPU
    'memory_fact': MACHINE_CONFIG['RAM']/16.0,          # How much memory to allocate. 1 works for 16Gb, so 0.8 showd be optimized for 12Gb.
    'niter_rig': 1,
    'splits_rig': 20,          # for parallelization split the movies in  num_splits chuncks across time
                               # if none all the splits are processed and the movie is saved
    'num_splits_to_process_rig': None} # intervals at which patches are laid out for motion correction  

for name in tqdm.tqdm(fnames):
    DoMotionCorrection(name, mc_dict)
    CleanMemmaps(name)

<h3>Module 2.5 (optional)</h3>

Pre-test of various values of <i>gSig</i> parameter, which is used in the Module 3 and corresponds to a typical radius of a neuron in pixels.<br/>You can play with this parameter but you can use the default value of gSig = 6 as well. <br/> Calculation may take a while, so be patient!

#### gSig

In [ ]:
fnames = askopenfilenames(title = 'Select files for gsig testing', initialdir = root, filetypes = [('TIFF files', '.tif')])

print(fnames[0])
Test_gSig_Range(fnames[0],default_gsig = 3, maxframes = 2000)  # maxframes is the amount of frames taken into account, by default the whole file is to be taken, which may be too slow for large files

#### min_corr and min_pnr

In [ ]:
%matplotlib ipympl
#fnames = glob(os.path.join(root, '*_MC.tif'))
fnames = askopenfilenames(title = 'Select files for corr image testing',initialdir = root, filetypes = [('TIFF files', '.tif')])

opt_gsig=5
test_min_corr_and_pnr(fnames[0], opt_gsig, start_frame=2000, end_frame=4000)

<h2>Module 3</h2>
Batch cnmf.<br/>All _MC.tif files in the root folder are to be automatically processed with CaImAn routine [Giovanucci et al., 2019] with the parameters below. Main parameters are gSig and gSiz for cell augmentation, then min_SNR as traces quality threshold. At the end, _estimates.pickle files are to be produced in the root folder. 

In [ ]:
gSig=5
min_corr = 0.85
min_pnr = 6
gSiz = gSig*4+1
cnmf_dict= {'fr': 20,                   # frame rate, frames per second (NOW RECALCULATED FOR EACH FILE FROM TIMESTAMP DATA)
            'decay_time': 1,            # typical duration of calcium transient 
            'method_init': 'corr_pnr',  # use this for 1 photon
            'K': None,                  # upper bound on number of components per patch, in general None
            'gSig': (gSig, gSig),       # gaussian HALF-width of a 2D gaussian kernel (in pixels), which approximates a neuron
            'gSiz': (gSiz, gSiz),       # maximal radius of a neuron in pixels
            'merge_thr': 0.8,           # merging threshold, max correlation allowed
            'p': 1,                     # order of the autoregressive system
            'tsub': 1,                  # downsampling factor in time for initialization
            'ssub': 1,                  # downsampling factor in space for initialization
            'rf': 40,                   # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
            'stride': 25,               # amount of overlap between the patches in pixels(keep it at least large as gSiz, i.e 4 times the neuron size gSig) 
            'only_init': True,          # set it to True to run CNMF-E
            'nb': 0,                    # number of background components (rank) if positive, else exact ring model with following settings: nb= 0: Return background as b and W, gnb=-1: Return full rank background B, gnb<-1: Don't return background
            'nb_patch': 0,              # number of background components (rank) per patch if nb>0, else it is set automatically
            'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
            'low_rank_background': None,           # None leaves background of each patch intact, True performs global low-rank approximation if gnb>0
            'update_background_components': True,  # sometimes setting to False improve the results
            'min_corr': min_corr,                        # min peak value from correlation image
            'min_pnr': min_pnr,                         # min peak to noise ratio from PNR image
            'normalize_init': False,               # just leave as is
            'center_psf': True,                    # leave as is for 1 photon
            'ssub_B': 2,                           # additional downsampling factor in space for background
            'ring_size_factor': 1.5,               # radius of ring is gSiz*ring_size_factor
            'del_duplicates': True,                # whether to remove duplicates from initialization
            'border_pix': 5,                       # number of pixels to not consider in the borders
            'min_SNR': 3,                          # adaptive way to set threshold on the transient size
            'rval_thr': 0.95,                      # threshold on space consistency           
            'use_cnn': False}                      # whether to use CNNs for event detection  

In [ ]:
fnames = glob(os.path.join(root, '*_MC.tif'))
#OR, alternatively, you can use manual file selection:
#fnames = askopenfilenames(title = 'Select files for batch cnmf', initialdir = root, filetypes = [('TIFF files', '.tif')])

for name in tqdm.tqdm(fnames):

    out_name = name[:-4-6] + f'_estimates.pickle'
    print(out_name)
    DoCNMF(name,
           cnmf_dict,
           out_name=out_name,
           verbose=True)
    
    #CleanMemmaps(name)  
    err_cnt = 0
    while err_cnt < 100:
        try:
            CleanMemmaps(name)
            break
        except PermissionError:
            time.sleep(1)
            err_cnt += 1
    print('CleanMemmaps attemps:', err_cnt)

<h2>Module 4</h2>
User inspection of cnmf results.<br/>
Btw, at this stage, previously saved timestamps are to be merged with cnmf results.
By running the section below, you will be prompted to select estimtes file with cnmf results and then to interactively examine detected units (you can select both spatial and temporal components), you can select, merge and delete them, also you can seed new neurons (by PointDrawTool) for further re-run of CNMF with saved seeds. Finally, you can save (by pressing 'Save Results') spatial and temporal components as .tif and traces.csv files, respectively. Spatial components (aka filters) are to be stored in a separate folder (*_filters).

In [ ]:
fname = askopenfilename(title = 'Select estimates file for examination',
                        initialdir = root,
                        filetypes = [('estimates files', '*estimates.pickle')])

bkapp_kwargs = {
    'start_frame': 0,          # start from this frame
    'end_frame': 90000,          # end at this frame
    'downsampling': 5,           # take every 'x' frame
    'fill_alpha': 1,       # selected neuron transparency
    'ns_alpha': 0.3,         # non-selected neuron transparency
    'line_width': 1.5,         # border width
    'cthr': 0.8,             # coutour_thr from caiman (% of signal inside a patch), affects patch size
    'line_alpha': 1,           # border transparency
    'trace_line_width': 2,       # trace line width
    'trace_alpha': 1,          # trace transparency
    'size': 900,                #height of the plot
    'button_width': 120,         # button width in pixels
    'verbose': 0,
    'enable_gpu_backend': 1,
    'emergency_mode': 0            #simple loading mode in case of slow performance due to large data amount 
}
ExamineCells(fname, default_fps=20, bkapp_kwargs=bkapp_kwargs)

<h2>Module 5 (optional)</h2>
Batch detection of significant calcium events. <br/>
INPUT: (timestamped) cnmf raw traces as *_traces.csv files<br/>
OUTPUT: detected events as *_spikes.csv files; pickles with events (cell-wise list of event-wise lists with dictionaries) and also, interactive .html plot with traces and events.

<h3> Fitting event detection</h3>
Each peak higher than the threshold value is fitted by bi-exponential calcium event waveform

In [ ]:
fnames = glob(root + '*traces.csv')
#OR, alternatively, you can use manual file selection:
#fnames = askopenfilenames(title = 'Select traces for event detection', initialdir = root, filetypes = [('traces files', '*traces.csv')])

sd_dict = {'thr': 4,        #threshold for peaks in Median Absolute Deviations (MADs)                    
           'sigma' : 7,     #smoothing parameter for peak detection, frames 
           'est_ton' : 0.5, #estimated event rising time, s 
           'est_toff' : 2,  #estimated event decay time, s 
           'draw_details': True} #whether to draw smoothed traces, peaks, pits and fits  
 
for name in fnames: 
    FitEvents(name, opts = sd_dict)

<h3> Wavelet event detection</h3>
Alternative way of event detection, using wavelet transform

In [ ]:
fnames = askopenfilenames(title = 'Select traces for event detection', initialdir = root, filetypes = [('traces files', '*traces.csv')])

wvt_param_dict = {'fps': 20,        # fps, frames                   
                  'sigma' : 8,      # smoothing parameter for peak detection, frames
                  'beta' : 2,       # Generalized Morse Wavelet parameter, FIXED
                  'gamma' : 3,      # Generalized Morse Wavelet parameter, FIXED
                  'eps': 10,         # spacing beween consecutive events, frames
                  'manual_scales': np.logspace(2.5,5.5,50, base=2),

                  # ridge filtering params
                  'scale_length_thr': 40,  # min number of scales where ridge is present thr, higher = less events. max=len(manual_scales)
                  'max_scale_thr': 7,      # index of a scale with max ridge intensity thr, higher = less events. < 5 = noise, > 20 = huge events
                  'max_ampl_thr': 0.075,    # max ridge intensity thr, higher = less events. < 5 = noise, > 20 = huge events
                  'max_dur_thr': 200,      # max event duration thr, higher = more events (but probably strange ones)
}
for fname in fnames:
    GetWaveletEvents(fname, opts = wvt_param_dict)

Also, just in case, you may draw existed pairs of traces and events right here:

In [ ]:
fnames = glob(root + '*traces.csv')
for name in fnames:
    DrawSpEvents(name, name.replace('traces','spikes'))

In [ ]:
#Draw two types of events simultaneously
fnames = glob(root + '*traces.csv')
for name in fnames:
    DrawSpEvents2(name, name.replace('traces','spikes'), name.replace('traces','wvt_spikes'))